# Micro Mortgages - Stacking
- Author: Oliver Mueller
- Last update: 26.01.2024

## Initialize notebook
Load required packages. Set up workspace, e.g., set theme for plotting and initialize the random number generator.

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import tree
#import graphviz as graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier


In [2]:
plt.style.use('fivethirtyeight')

## Problem description

In India, there are about 20 million home loan (mortgage) aspirants
working in the informal sector:

- Monthly income between INR 20,000-25,000 (\$ 325-400)
- Typically no formal accounts and documents (e.g., tax returns, income proofs, bank statements)
- Often use services of money lenders with interest rates between 30 and 60% per annum

Providing mortgages to this group of customers requires to quickly and
efficiently assess their creditworthiness. Due to a lack of formal
documents and objective data, most financial institutions perform
interview-based processes to decide about these loan requests:

Strength of the current process:

-   Interview-based field assessment

-   Relaxation of document requirements

Weaknesses of the current process:

-   Costly (total transaction costs as high as 30% of loan volume)

-   Subjective judgments; depends on individual skills and motivations

-   Low reliability across branches and credit officers

-   Risk of corruption and fraud

## Load data

Load training data from CSV file.

In [3]:
data = pd.read_csv('data/micromortgage.csv')

In [4]:
data.head()

,ID,Decision,Build_Selfcon,Tier,Accommodation_Class,Loan_Type,Gender,Employment_Type,Doc_Proof_Inc,Marital_Status,...,LoanReq,Term,Dwnpay,BankSave,CalcEmi,IIR,IAR,FOIR,LTV,LVR
0,FBD-E2B0-588300,1,Self Contruction,2,Non_Rented,Home_Loan,Female,Salaried,N,Married,...,780000,180,670000,0,12004.230470,34.999797,45.000114,34.999797,80.000000,54.000000
1,GUJ-A79X-831476,0,Self Contruction,1,Non_Rented,Home_Loan,Female,Self_Employed,N,Married,...,800000,180,470000,0,12312.030270,49.248121,75.533928,49.248121,62.992126,62.992126
2,SHB-947O-759226,1,Self Contruction,3,Rented,Home_Loan,Female,Salaried,N,Married,...,480000,120,120000,300000,8342.290039,41.999144,79.998946,41.999144,78.999992,80.000000
3,SHB-7S3I-679761,1,Self Contruction,3,Non_Rented,Home_Loan,Female,Self_Employed,N,Married,...,300000,180,95000,0,4617.009766,30.999126,84.996498,30.999126,20.000000,76.000000
4,VAD-BPKZ-551476,0,Self Contruction,2,Non_Rented,Home_Loan,Female,Self_Employed,N,Married,...,1000000,180,375000,0,15390.040040,45.000117,57.999020,45.000117,73.000001,73.000000


## Prepare data

In [5]:
data = data.drop(['ID'], axis=1)
data["Tier"] = data["Tier"].apply(lambda x: "T"+str(x))

In [6]:
X = data.drop("Decision", axis=1)
y = data["Decision"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
categorical_features = X_train.select_dtypes(include='object').columns
numerical_features = X_train.select_dtypes(exclude='object').columns

In [8]:
categorical_features

Index(['Build_Selfcon', 'Tier', 'Accommodation_Class', 'Loan_Type', 'Gender',
       'Employment_Type', 'Doc_Proof_Inc', 'Marital_Status', 'Employer_Type',
       'Education_Class', 'Mode_of_origin_class'],
      dtype='object')

In [9]:
numerical_features

Index(['eom_25', 'OldEmi_d', 'BankSave_d', 'Age', 'YrsAdd', 'YrsJob', 'Expen',
       'TotInc', 'DispInc', 'MarVal', 'OldEmi', 'LoanReq', 'Term', 'Dwnpay',
       'BankSave', 'CalcEmi', 'IIR', 'IAR', 'FOIR', 'LTV', 'LVR'],
      dtype='object')

In [10]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(X_train[categorical_features])

X_train_cat = enc.transform(X_train[categorical_features])
X_test_cat = enc.transform(X_test[categorical_features])

X_train_cat = pd.DataFrame(X_train_cat, columns=enc.get_feature_names_out(categorical_features))
X_test_cat = pd.DataFrame(X_test_cat, columns=enc.get_feature_names_out(categorical_features))

In [11]:
X_train_cat.head()

,Build_Selfcon_Builder,Build_Selfcon_Self Contruction,Tier_T1,Tier_T2,Tier_T3,Accommodation_Class_Non_Rented,Accommodation_Class_Rented,Loan_Type_Home_Improvement,Loan_Type_Home_Loan,Gender_Female,...,Employer_Type_Business,Employer_Type_Corporate,Employer_Type_Govt,Employer_Type_Ind/SmallBus,Education_Class_GRADUATE+,Education_Class_U10,Education_Class_UNDERGRADUATE,Mode_of_origin_class_Others,Mode_of_origin_class_Own database field visit,Mode_of_origin_class_Reference
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [12]:
scaler = StandardScaler()
scaler.fit(X_train[numerical_features]) 

X_train_num = scaler.transform(X_train[numerical_features])
X_test_num = scaler.transform(X_test[numerical_features])

X_train_num = pd.DataFrame(X_train_num, columns=numerical_features)
X_test_num = pd.DataFrame(X_test_num, columns=numerical_features)

In [13]:
X_train_num.head()

,eom_25,OldEmi_d,BankSave_d,Age,YrsAdd,YrsJob,Expen,TotInc,DispInc,MarVal,...,LoanReq,Term,Dwnpay,BankSave,CalcEmi,IIR,IAR,FOIR,LTV,LVR
0,-0.523557,-0.389071,-0.354345,0.492436,1.452931,1.148835,1.758042,0.636953,-0.003758,0.468872,...,0.911333,0.552998,0.854405,-0.174566,0.787476,-0.607180,0.156632,-0.783101,-0.145868,-0.439966
1,-0.523557,-0.389071,-0.354345,1.080875,-0.838288,-1.119960,-0.663347,-0.405469,-0.199890,0.176403,...,0.567547,0.552998,0.517944,-0.174566,0.437153,1.379933,0.429896,1.167426,-0.055918,-0.349566
2,1.910013,-0.389071,-0.354345,-0.331378,-0.360951,-0.867872,-0.680420,0.432506,0.829239,0.211792,...,1.770797,0.552998,-0.183017,-0.174566,1.663286,0.433728,-0.708569,0.238642,0.843581,0.554433
3,-0.523557,-0.389071,-0.354345,-0.096002,0.498256,0.518614,-0.245413,-0.054211,0.042832,0.040139,...,0.051868,0.552998,1.754439,-0.174566,-0.088331,-0.338312,-0.709183,-0.519184,-0.309925,-0.604844
4,-0.523557,2.570226,-0.354345,0.610124,-0.170016,1.779055,2.001719,0.714879,-0.016972,1.914943,...,1.770797,0.552998,3.097480,-0.174566,1.663286,-0.039460,0.976423,1.594633,-0.730541,-1.027565


In [14]:
X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)

## LASSO

In [15]:
model_lasso_cv = lambda: GridSearchCV(
                estimator=LogisticRegression(random_state=42),
                param_grid={
                        'C': np.logspace(-4, 4, 10)
                    }, cv=5, n_jobs=-1
                )

In [16]:
model_lasso_tuned = model_lasso_cv()
model_lasso_tuned.fit(X_train, y_train)

/Users/oliver/miniconda3/envs/prodok/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/oliver/miniconda3/envs/prodok/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])})

In [17]:
pred_lasso_label = model_lasso_tuned.predict(X_test)
pred_lasso_proba = model_lasso_tuned.predict_proba(X_test)[:,1]

In [18]:
print(classification_report(y_test, pred_lasso_label))

              precision    recall  f1-score   support

           0       0.81      0.31      0.45        55
           1       0.84      0.98      0.90       197

    accuracy                           0.83       252
   macro avg       0.82      0.64      0.67       252
weighted avg       0.83      0.83      0.80       252



In [19]:
roc_auc_score(y_test, pred_lasso_proba)

0.8151361329026303

## Random Forest

In [20]:
model_rf_cv = lambda: GridSearchCV(
                estimator=RandomForestClassifier(random_state=42),
                param_grid={
                        'n_estimators': [100, 200, 300],
                        'max_depth': [3, 5, None],
                        'min_samples_leaf': [2, 5, 10]
                    }, cv=5, n_jobs=-1
                )

In [21]:
model_rf_tuned = model_rf_cv()
model_rf_tuned.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [3, 5, None],
                         'min_samples_leaf': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

In [22]:
pred_rf_label = model_rf_tuned.predict(X_test)
pred_rf_proba = model_rf_tuned.predict_proba(X_test)[:,1]

In [23]:
print(classification_report(y_test, pred_rf_label))

              precision    recall  f1-score   support

           0       1.00      0.31      0.47        55
           1       0.84      1.00      0.91       197

    accuracy                           0.85       252
   macro avg       0.92      0.65      0.69       252
weighted avg       0.87      0.85      0.82       252



In [24]:
roc_auc_score(y_test, pred_rf_proba)

0.8487309644670051

## Stack models

Here, we will use a simple stacking approach by combining the predictions of the LASSO and Random Forest models with equal weight.

In [25]:
pred_final_proba = (pred_lasso_proba + pred_rf_proba) / 2

In [26]:
roc_auc_score(y_test, pred_final_proba)

0.8516843562528842